In [ ]:
import pandas as pd
import numpy as np
import random
import os
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
os.environ['PYTHONHASHSEED'] = str(RANDOM_SEED)

import tensorflow as tf
tf.random.set_seed(RANDOM_SEED)

X_scaled = pd.read_csv('../data/processed/X_scaled.csv')
y = pd.read_csv('../data/processed/y.csv')['outcome']

print(f"Loaded data shape: {X_scaled.shape}")
print(f"Target shape: {y.shape}")

# 1. Feature Engineering

In [ ]:
# Check for missing values and clean data
print(f"Missing values in y: {y.isna().sum()}")
print(f"Outcome distribution:\n{y.value_counts()}")
print(f"Outcome unique values: {y.unique()}")

# Remove rows with missing outcomes from BOTH X and y
mask = ~y.isna()
X_scaled_clean = X_scaled[mask]
y_clean = y[mask]

print(f"\nCleaned data shapes:")
print(f"X: {X_scaled_clean.shape}, y: {y_clean.shape}")


# 2. Model Development

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled_clean, y_clean, test_size=0.2, random_state=42, stratify=y_clean)

print(f"Train set: {X_train.shape}")
print(f"Test set: {X_test.shape}")
print(f"Train outcome distribution:\n{pd.Series(y_train).value_counts()}")
print(f"Test outcome distribution:\n{pd.Series(y_test).value_counts()}")


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

print("Training Logistic Regression...")
lr_model = LogisticRegression(max_iter=1000, random_state=42) 
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)

print("\nLogistic Regression Results:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_lr):.4f}")
print(classification_report(y_test, y_pred_lr))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

print("Training Random Forest...")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

print("\nRandom Forest Results:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")
print(classification_report(y_test, y_pred_rf))

In [ ]:
import xgboost as xgb

print("Training XGBoost...")
xgb_model = xgb.XGBClassifier(n_estimators=100, random_state=42, eval_metric='mlogloss')
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

print("\nXGBoost Results:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_xgb):.4f}")
print(classification_report(y_test, y_pred_xgb))

In [ ]:
n_classes = len(np.unique(y_train))
print(f"Number of classes: {n_classes}")

In [ ]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

print("Training Neural Network...")

# two hidden layers with dropout to prevent overfitting
nn_model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')  
])
nn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
nn_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=0)
y_pred_nn = (nn_model.predict(X_test, verbose=0) > 0.5).astype(int).flatten()

print("\nNeural Network Results:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_nn):.4f}")
print(classification_report(y_test, y_pred_nn))

In [ ]:
results = {
    'Logistic Regression': accuracy_score(y_test, y_pred_lr),
    'Random Forest': accuracy_score(y_test, y_pred_rf),
    'XGBoost': accuracy_score(y_test, y_pred_xgb),
    'Neural Network': accuracy_score(y_test, y_pred_nn)
}

print("\n" + "-"*50)
print("MODEL COMPARISON")
print("-"*50)
for model, acc in sorted(results.items(), key=lambda x: x[1], reverse=True):
    print(f"{model:20s}: {acc:.4f}")
print("-"*50)

# 3. Risk Stratification

Convert binary predictions into three-class risk categories based on predicted probabilities for better clinical interpretation.

In [ ]:
def predict_risk_category(probabilities, low_threshold=0.3, high_threshold=0.7):
    """
    Convert predicted probabilities to risk categories.
    
    Args:
        probabilities: Array of predicted probabilities for positive class
        low_threshold: Threshold below which risk is 'Low' (default: 0.3)
        high_threshold: Threshold above which risk is 'High' (default: 0.7)
    
    Returns:
        Array of risk categories: 0 (Low), 1 (Medium), 2 (High)
    """
    risk_categories = np.zeros(len(probabilities), dtype=int)
    risk_categories[(probabilities >= low_threshold) & (probabilities < high_threshold)] = 1
    risk_categories[probabilities >= high_threshold] = 2
    
    return risk_categories

print("Extracting predicted probabilities...\n")

prob_lr = lr_model.predict_proba(X_test)[:, 1]
risk_lr = predict_risk_category(prob_lr)

prob_rf = rf_model.predict_proba(X_test)[:, 1]
risk_rf = predict_risk_category(prob_rf)

prob_xgb = xgb_model.predict_proba(X_test)[:, 1]
risk_xgb = predict_risk_category(prob_xgb)

prob_nn = nn_model.predict(X_test, verbose=0).flatten()
risk_nn = predict_risk_category(prob_nn)

print("Risk stratification complete!")

In [ ]:
# displaying risk distribution for each model
risk_labels = ['Low Risk', 'Medium Risk', 'High Risk']

models_risk = {
    'Logistic Regression': risk_lr,
    'Random Forest': risk_rf,
    'XGBoost': risk_xgb,
    'Neural Network': risk_nn
}

print("\n" + "-"*60)
print("RISK CATEGORY DISTRIBUTION")
print("-"*60)

for model_name, risk_pred in models_risk.items():
    print(f"\n{model_name}:")
    print("-" * 40)
    unique, counts = np.unique(risk_pred, return_counts=True)
    for risk_level, count in zip(unique, counts):
        percentage = (count / len(risk_pred)) * 100
        print(f"  {risk_labels[risk_level]:15s}: {count:4d} ({percentage:5.1f}%)")

print("-"*60)

In [ ]:
y_test_array = np.array(y_test)

from sklearn.metrics import confusion_matrix, classification_report

print("\n" + "-"*60)
print("RISK STRATIFICATION PERFORMANCE (XGBoost)")
print("-"*60)

risk_performance = pd.DataFrame({
    'Risk Category': risk_labels,
    'Count': [np.sum(risk_xgb == i) for i in range(3)],
    'Actual Deaths': [np.sum(y_test_array[risk_xgb == i]) for i in range(3)],
    'Death Rate (%)': [np.mean(y_test_array[risk_xgb == i]) * 100 if np.sum(risk_xgb == i) > 0 else 0 for i in range(3)]
})

print(risk_performance.to_string(index=False))
print("-"*60)

print("\nInterpretation:")
print("- Low Risk: Should have low death rate")
print("- Medium Risk: Intermediate death rate (uncertain cases)")
print("- High Risk: Should have high death rate")

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Predicted Probability Distributions by Risk Category', fontsize=16, fontweight='bold')

models_prob = [
    ('Logistic Regression', prob_lr, risk_lr),
    ('Random Forest', prob_rf, risk_rf),
    ('XGBoost', prob_xgb, risk_xgb),
    ('Neural Network', prob_nn, risk_nn)
]

for idx, (name, probs, risks) in enumerate(models_prob):
    ax = axes[idx // 2, idx % 2]
    
    # plotting histogram for each risk category
    colors = ['green', 'orange', 'red']
    for risk_level in range(3):
        mask = risks == risk_level
        if np.sum(mask) > 0:
            ax.hist(probs[mask], bins=20, alpha=0.6, label=risk_labels[risk_level], 
                   color=colors[risk_level], edgecolor='black')
    
    ax.axvline(x=0.3, color='blue', linestyle='--', linewidth=1.5, label='Low/Med threshold')
    ax.axvline(x=0.7, color='purple', linestyle='--', linewidth=1.5, label='Med/High threshold')
    
    ax.set_xlabel('Predicted Probability', fontsize=11)
    ax.set_ylabel('Frequency', fontsize=11)
    ax.set_title(name, fontsize=12, fontweight='bold')
    ax.legend(loc='upper right', fontsize=9)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Evaluate 3-class risk prediction performance

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print("\n" + "-"*60)
print("3-CLASS RISK STRATIFICATION PERFORMANCE")
print("-"*60)

models_risk_eval = {
    'Logistic Regression': risk_lr,
    'Random Forest': risk_rf,
    'XGBoost': risk_xgb,
    'Neural Network': risk_nn
}

for model_name, risk_pred in models_risk_eval.items():
    print(f"\n{model_name}:")
    print("-" * 50)
    
    # create a simplified 2-class version for comparison (Low vs High)
    # treat Medium as Low risk for evaluation purposes
    risk_binary = risk_pred.copy()
    risk_binary[risk_binary == 1] = 0  # Medium -> Low for comparison
    risk_binary[risk_binary == 2] = 1  # High -> High
    
    print(classification_report(y_test_array, risk_binary, 
                                target_names=['Low/Medium Risk', 'High Risk']))

In [ ]:
import seaborn as sns

fig, ax = plt.subplots(1, 1, figsize=(8, 6))

# create binary version: Low/Medium vs High
risk_binary_xgb = risk_xgb.copy()
risk_binary_xgb[risk_binary_xgb == 1] = 0
risk_binary_xgb[risk_binary_xgb == 2] = 1

cm = confusion_matrix(y_test_array, risk_binary_xgb)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
            xticklabels=['Predicted: Low/Med', 'Predicted: High'],
            yticklabels=['Actual: Survived', 'Actual: Died'])

ax.set_title('Risk Stratification Confusion Matrix (XGBoost)', fontsize=14, fontweight='bold')
ax.set_ylabel('Actual Outcome', fontsize=12)
ax.set_xlabel('Predicted Risk Category', fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score

risk_results = {}

for model_name, risk_pred in models_risk_eval.items():
    # convert to binary for fair comparison
    risk_binary = risk_pred.copy()
    risk_binary[risk_binary == 1] = 0  # Medium -> Low
    risk_binary[risk_binary == 2] = 1  # High
    
    acc = accuracy_score(y_test_array, risk_binary)
    risk_results[model_name] = acc

print("\n" + "-"*60)
print("RISK STRATIFICATION MODEL COMPARISON")
print("-"*60)
for model, acc in sorted(risk_results.items(), key=lambda x: x[1], reverse=True):
    print(f"{model:20s}: {acc:.4f}")
print("-"*60)

# compare with original binary predictions
print("\nComparison with Original Binary Predictions:")
print("-" * 60)
for model in results.keys():
    print(f"{model:20s} - Binary: {results[model]:.4f} | Risk: {risk_results[model]:.4f}")
print("-" * 60)

### dataset is heavily imbalanced. will apply smote in the next file.